# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data=pd.read_csv("../output_data/cities.csv")
cities_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,talnakh,69.49,88.40,60.80,67.0,34.0,8.95,RU,1596036971
1,bluff,-46.60,168.33,50.32,91.0,100.0,13.80,NZ,1596036919
2,port alfred,-33.59,26.89,60.57,70.0,0.0,14.52,ZA,1596036937
3,hobart,-42.88,147.33,48.20,61.0,13.0,12.75,AU,1596036878
4,qaanaaq,77.48,-69.36,42.49,73.0,70.0,2.13,GL,1596036972


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)
locations=cities_data[["Lat","Lng"]]
humidity_lvl=cities_data["Humidity"]

In [4]:
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations,weights=humidity_lvl,dissipating=False,point_radius=2,
                              max_intensity=humidity_lvl.max())
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_data.head()
top_spots=cities_data.loc[cities_data["Max Temp"] >= 65]
top_spots=cities_data.loc[cities_data["Max Temp"] <= 85]
top_spots=cities_data.loc[cities_data["Wind Speed"] <= 10]
top_spots.reset_index(drop=True, inplace=True)
top_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,talnakh,69.49,88.40,60.80,67.0,34.0,8.95,RU,1596036971
1,qaanaaq,77.48,-69.36,42.49,73.0,70.0,2.13,GL,1596036972
2,lavrentiya,65.58,-171.00,47.59,90.0,95.0,4.81,RU,1596036972
3,khatanga,71.97,102.50,54.37,72.0,6.0,5.64,RU,1596036973
4,oksfjord,70.24,22.35,69.80,56.0,100.0,9.17,NO,1596036974
...,...,...,...,...,...,...,...,...,...
405,mosquera,4.71,-74.23,60.80,63.0,75.0,6.93,CO,1596037160
406,caravelas,-17.71,-39.25,79.75,57.0,0.0,7.74,BR,1596037161
407,los andes,-32.83,-70.60,63.00,61.0,0.0,2.24,CL,1596037161
408,matagami,49.75,-77.63,62.60,93.0,75.0,4.41,CA,1596037161


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
top_spots.loc[:,"Hotel"]=""
top_spots.head()
sample_spots=top_spots.sample(n=10)
sample_spots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
208,sechura,-5.56,-80.82,69.42,59.0,30.0,7.45,PE,1596037070,
309,itoman,26.12,127.67,84.20,83.0,20.0,9.17,JP,1596037117,
70,liverpool,53.41,-2.98,66.00,63.0,92.0,8.05,GB,1596036991,
40,hermanus,-34.42,19.23,63.00,73.0,0.0,4.00,ZA,1596036844,
334,awjilah,29.11,21.29,99.99,15.0,0.0,6.73,LY,1596037127,
8,mataura,-46.19,168.86,47.88,91.0,100.0,3.27,NZ,1596036975,
91,kumluca,36.37,30.29,90.39,46.0,0.0,6.04,TR,1596037012,
87,shenzhen,22.55,114.07,86.00,74.0,96.0,8.95,CN,1596037011,
325,finschhafen,-6.60,147.87,79.21,78.0,11.0,8.86,PG,1596037123,
243,salym,60.06,71.48,52.03,84.0,74.0,2.89,RU,1596037085,


In [7]:
params = {
    "key": gkey,
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel"
}
for index,row in sample_spots.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    results=requests.get(base_url,params=params)
    results=results.json()
    try:
        sample_spots.loc[index,"Hotel"]=results["results"][0]["name"]
    except (KeyError, IndexError):
        print("Location missing...skipping")
print("-"*30)
print("Finished")

Location missing...skipping
Location missing...skipping
------------------------------
Finished


In [9]:
sample_spots["Hotel"].replace("",np.nan,inplace=True)
sample_spots.dropna(subset=["Hotel"], inplace=True)
sample_spots.reset_index(drop=True, inplace=True)
sample_spots.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,sechura,-5.56,-80.82,69.42,59.0,30.0,7.45,PE,1596037070,Hotel Amanecer - Sechura
1,itoman,26.12,127.67,84.20,83.0,20.0,9.17,JP,1596037117,Private Villa Yui Terrace
2,liverpool,53.41,-2.98,66.00,63.0,92.0,8.05,GB,1596036991,DoubleTree by Hilton Hotel & Spa Liverpool
3,hermanus,-34.42,19.23,63.00,73.0,0.0,4.00,ZA,1596036844,One Marine Drive Boutique Hotel
4,mataura,-46.19,168.86,47.88,91.0,100.0,3.27,NZ,1596036975,Southland Organic Farmstay
5,kumluca,36.37,30.29,90.39,46.0,0.0,6.04,TR,1596037012,TurkuazKöy
6,shenzhen,22.55,114.07,86.00,74.0,96.0,8.95,CN,1596037011,Hotel Novotel Shenzhen Watergate
7,salym,60.06,71.48,52.03,84.0,74.0,2.89,RU,1596037085,Gostinitsa V Salyme


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in sample_spots.iterrows()]
hotel_locations = sample_spots[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
hotel_layer=gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))